In [1]:
from itertools import product
from pysm3.models import Model, CMBLensed

In [2]:
ps_c1_fp = "demo_powerspectra/c1_ps.dat"
delensing_ells_c1_fp = "demo_powerspectra/c1_delensing_ells.dat"

# Problem

Running CMBLensed with `apply_delens=False` (default) works as expected.

In [3]:
CMBLensed(nside=128, cmb_spectra=ps_c1_fp, delensing_ells=delensing_ells_c1_fp, apply_delens=False)

Running CMBLensed with `apply_delens=True` causes a ValueError.

In [4]:
try:
    CMBLensed(nside=128, cmb_spectra=ps_c1_fp, delensing_ells=delensing_ells_c1_fp, apply_delens=True)
except ValueError as e:
    print(e)

operands could not be broadcast together with shapes (2199,) (2,) 


# Diagnosing

The delensing ells are transposed. Note that in the following output, the long axis goes from 0 to 1.

In [5]:
a = CMBLensed(nside=128, cmb_spectra=ps_c1_fp, delensing_ells=delensing_ells_c1_fp, apply_delens=False)
a.cmb_spectra.shape, a.delensing_ells.shape

((8, 2199), (2199, 2))

Changing the way the delensing ells are read (adding `unpack=True`) aligns the matrices as the code expects.

In [6]:
class SurrogateCMBLensed(Model):
    def __init__(self, ps_file, apply_delens, nside=2048):
        super().__init__(nside=nside)
        self.nside = nside
        self.cmb_seed = 8675309
        self.cmb_spectra = self.read_txt(ps_file, unpack=True)
        self.apply_delens = apply_delens
        self.delensing_ells = self.read_txt(delensing_ells_c1_fp, unpack=True)  # Fix here!
        self._map = None

Now the axes line up.

In [7]:
b = SurrogateCMBLensed(ps_c1_fp, True, nside=128)
b.cmb_spectra.shape, b.delensing_ells.shape

((8, 2199), (2, 2199))

And the code runs.

In [8]:
b.run_taylens = CMBLensed.run_taylens  # Patch in the method
b._map = b.run_taylens(b)  # 4.5 minutes at nside=2048; 1 second at nside=128

# NOT COMPLETE!

I think I need to account for the presence or absence of L=0 and L=1, following Issue 212.